<a href="https://www.kaggle.com/code/ayushs9020/a-simple-beginner-friendly-approach-kaggle-llm?scriptVersionId=137450322" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF6C00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF6C00">Kaggle LLM</p>

<div style="border-radius:10px; border:#FF6C00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
The $Kaggle - LLM$ $Science$ $Exam$ is a `competition` that challenges to `answer difficult science-based questions` written by a `Large Language Model` $(LLM)$. The `Goal` of the competition is to help `researchers better understand` the `ability of LLMs` to test themselves, and the `potential of LLMs` that can be run in resource-constrained environments.

The `dataset` for the competition was generated by giving `gpt3.5 snippets` of text on a range of `scientific topics pulled` from `Wikipedia`, and asking it to `write a multiple choice question` (with a known answer), then `filtering out easy questions`.

`Participants` in the competition are asked to `develop an LLM` that can `answer the questions` in the dataset `as accurately as possible`. The competition is scored using the `average precision` at `cutoff k metric`, where $k$ is the `number of predictions` made for each question.

An estimations shays that the `largest models` run on `Kaggle` are around $10$ $Billion$ $Parameters$, whereas `gpt3.5 clocks` in at $175$ $Billion$ $Parameters$. If a `question-answering model can ace` a test written by a `question-writing model` more than $10$ `times its size`, this would be a genuinely `interesting result`; on the `other hand` if a `larger model can effectively` `stump a smaller one`, this has `compelling implications` on the `ability of LLMs` to benchmark and test themselves.

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FFFF; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FFFF">1 | Approach</p>

<div style="border-radius:10px; border:#00FFFF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
* Replace the values in `answer` with actual `textual answers`
* Remove the columns `["id", "A" , "B" . "C" . "D" , "E"]`
* Make Embeddings for both of the columns
* Train a Simple NN that will predict answer when given the prompt
* Use the distance between $2$ vectors as a loss function
* Watch the results

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF1493; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF1493">2 | Data 📊</p>

In [1]:
import pandas as pd

<div style="border-radius:10px; border:#FF1493 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Lets just focus on the training data 

In [2]:
train = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
train.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF7C; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF7C">3 | Data Preperations 📈</p>

In [3]:
import tqdm

<div style="border-radius:10px; border:#00FF7C solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now we will replace the `answers` with actual text data

In [4]:
for index in tqdm.tqdm(range(200) , total = 200):
    train["answer"][index] = train[train["answer"][index]][index]
    
train.head()

  0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_20/2369363817.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["answer"][index] = train[train["answer"][index]][index]
100%|██████████| 200/200 [00:00<00:00, 3373.32it/s]


,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,MOND is a theory that reduces the discrepancy ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,The triskeles symbol was reconstructed as a fe...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


<div style="border-radius:10px; border:#00FF7C solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Lets remove the remaining columns as they are of no use to us for now 

In [5]:
train.drop(["id" , "A" , "B" , "C" , "D" , "E"] , axis = 1 , inplace = True)

train.head()

,prompt,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the discrepancy ...
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF0000; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF0000">4 | Tokenization 🐱‍👤</p>

In [6]:
from transformers import AutoTokenizer , AutoModel
import torch

<div style="border-radius:10px; border:#FF0000 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We will use `RoBERTa` for gathering `Embeddings`. We might change this in further versions
    
## $RoBERTa$
    
$RoBERTa$ $Robustly$ $Optimized$ $BERT$ $Pretraining$ $Approach$ is a $Natural$ $Language$ $Processing$ $(NLP)$ model that was proposed in $2019$ by `Yinhan` Liu et al. It is a `reimplementation` of $BERT$ ($Bidirectional$ $Encoder$ $Representations$ from $Transformers$) with some `modifications` to the key `hyperparameters` and `minor embedding tweaks`. These modifications led to `significant performance gains` on a number of NLP tasks. $RoBERTa$ is based on the `transformer architecture`, which is a `Neural Network Architecture` that is particularly well-suited for NLP tasks. The transformer architecture uses `self-attention` to learn `long-range dependencies` between words in a sentence. This allows $RoBERTa$ to learn more `contextual representations` of words, which is important for many NLP tasks.

$RoBERTa$ is trained on a `massive dataset` of text and code. The dataset consists of `books`/`articles`/`code`. The dataset is `preprocessed` using `Byte-Level` `BPE` `(Byte Pair Encoding)`, which is a technique for tokenizing text into smaller units.

$RoBERTa$ is trained using a `Masked Language Modeling` ($MLM$) objective. In the MLM objective, some of the words in a `sentence are masked`, and the model is then trained to `predict the masked words`. This helps the model to `learn the contextual representations` of words.

In [7]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModel.from_pretrained("roberta-base").to("cuda")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 tokenizer = AutoTokenizer.from_pretrained("roberta-base")                                    │
│ ❱ 2 model = AutoModel.from_pretrained("roberta-base").to("cuda")                                 │
│   3                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1902 in to                │
│                                                                                                  │
│   1899 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1900 │   │   │   )                                                                             │
│   1901 │   │   else:                                                                             │
│ ❱ 1902 │   │   │   return super().to(*args, **kwargs)                                            │
│   1903 │                                                                                         │
│   1904 │   def half(self, *args):                                                                │
│   1905 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1145 in to                    │
│                                                                                                  │
│   1142 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1143 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1144 │   │                                                                                     │
│ ❱ 1145 │   │   return self._apply(convert)                                                       │
│   1146 │                                                                                         │
│   1147 │   def register_full_backward_pre_hook(                                                  │
│   1148 │   │   self,                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:797 in _apply                 │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    800 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:797 in _apply                 │
│                                                                                                  │
│    794 │                                                   

<div style="border-radius:10px; border:#FF0000 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will make embeddings from textual data 

In [ ]:
for index in tqdm.tqdm(range(200)):
    for columns in train.columns:
        train[columns][index] = model(tokenizer(train[columns][index] , return_tensors = "pt")["input_ids"].to("cuda"))[0][0][0]
        torch.cuda.empty_cache()
train.head()

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#0047AB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #0047AB">5 | Training Arguments 🤐</p>

<div style="border-radius:10px; border:#0047AB solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $Loss$ $Function$
    
We will try to use a normal loss function this time, We might update the loss function further, seeing the results
    
A loss function like this 
    
```
def loss(preds , targets):
    val = 0
    for index in range(768):
        val += (preds[index] ** 2) - (targets[index] ** 2)

    return val ** (1/2)
```
gives higher $10$ times higher losses

In [ ]:
def loss(preds , targets):

    return torch.sum(preds - targets)

<div style="border-radius:10px; border:#0047AB solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $Optimizer$

We will use Adam optimizer

In [ ]:
optim = torch.optim.Adam(model.parameters())

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF00">6 | Training Loop ➰</p>

<div style="border-radius:10px; border:#00FF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now we will start the training loop. To save time for this run, We have made many small tunes, which might counter the model efficiency a little bit.
    
I am not using Kaggle GPU, but have imported the results from Wanbd 
    
```
losses = []
for x , y in tqdm.tqdm(zip(train["prompt"] , train["answer"]) , total = 200):
    x = torch.tensor(x , dtype = torch.long).to("cuda")
    y = torch.tensor(y , dtype = torch.float32).to("cuda")
    
    x = x.reshape(shape = (1 , x.shape[0]))
    
    if x.shape[1] > 512: x = x[: , :512]

    pred = model(x)[0][0][0]

    loss_fun = loss(pred , y)
    losses.append(loss_fun)
    
    torch.cuda.empty_cache()

    optim.step
```

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#006600; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #003300">7 | Results Visualization 💪</p>

In [ ]:
from IPython.display import IFrame

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    

loss	▃▅▅▂▂▃▇▁▂▂▇▁▄▂▂▂▆▇▁▂▅▂▃▃▇▄█▂▂▂▂▂▃▂▃▂▃▄▁▁

Run summary:

loss	-0.41186
    
Now lets see how our model performed

In [ ]:
IFrame("https://wandb.ai//ayushsinghal659/Kaggle%20LLM%20%7C%20Simple%20Approach/reports/Simple-Approach-Kaggle-LLM--Vmlldzo0OTMxMzIz" , 1000 , 400)

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#AEFF32; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #AEFF32">8 | TO DO LIST 📝</p>

<div style="border-radius:10px; border:#AEFF32 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

* $TO$ $DO$ $1$ $:$ $MAKE$ $BETTER$ $LOSS$ $FUNCTION$
* $TO$ $DO$ $1$ $:$ $MAKE$ $PROPER$ $NN$
* $TO$ $DO$ $1$ $:$ $DANCE$

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00F900; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00F900">9 | Ending 🏁</p>

<div style="border-radius:10px; border:#00F900 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
**THAT IT FOR TODAY GUYS**

**WE WILL GO DEEPER INTO THE DATA IN THE UPCOMING VERSIONS**

**PLEASE COMMENT YOUR THOUGHTS, HIHGLY APPRICIATED**

**DONT FORGET TO MAKE AN UPVOTE, IF YOU LIKED MY WORK $:)$**
    
<img src = "https://i.imgflip.com/19aadg.jpg">
    
**PEACE OUT $!!!$**